In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-05 19:44:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2022-03-05 19:44:45 (1.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://dbaek-bucket.s3.amazonaws.com/amazon_reviews_us_Office_Products_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [7]:
#Filter and create new DataFrame with all rows where total voates >=20
above_20_df = df.filter(df.total_votes >= 20)
above_20_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11212836|R243UXMMSSZVS6|B00C2MBMNS|     934393421|Lion 2-Pocket Pla...| Office Products|          2|          254|        254|   N|                Y|      Disappointing!|We ordered about ...| 2015-08-31|
|         US|   23384746|R2P92EHR0S5GBB|B00C47U5OI|     712795023|24pcs Vitamin pil...| Office Products|          1|    

In [8]:
#Filter the df with helpful_votes / total_votes >= 0.5
ratio_df = above_20_df.filter((above_20_df.helpful_votes / above_20_df.total_votes) >= 0.5)
ratio_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11212836|R243UXMMSSZVS6|B00C2MBMNS|     934393421|Lion 2-Pocket Pla...| Office Products|          2|          254|        254|   N|                Y|      Disappointing!|We ordered about ...| 2015-08-31|
|         US|   23384746|R2P92EHR0S5GBB|B00C47U5OI|     712795023|24pcs Vitamin pil...| Office Products|          1|    

In [12]:
#vine == 'Y'
vine_df = ratio_df.filter(ratio_df.vine == 'Y')
vine_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35360512|R3JIGR122X50ZV|B00ZI7MI96|     592756103|Scotch Thermal La...| Office Products|          5|         2280|       2314|   Y|                N|Here’s what’s REA...|You’re reading th...| 2015-08-21|
|         US|   50120605|R3KV8P8WLD2KKC|B00ZI7MI96|     592756103|Scotch Thermal La...| Office Products|          5|    

In [13]:
#vine == 'N'
novine_df = ratio_df.filter(ratio_df.vine == 'N')
novine_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11212836|R243UXMMSSZVS6|B00C2MBMNS|     934393421|Lion 2-Pocket Pla...| Office Products|          2|          254|        254|   N|                Y|      Disappointing!|We ordered about ...| 2015-08-31|
|         US|   23384746|R2P92EHR0S5GBB|B00C47U5OI|     712795023|24pcs Vitamin pil...| Office Products|          1|    

In [30]:
#total # of reviews for paid
total_paid_reviews = vine_df.count()
total_paid_reviews

969

In [18]:
#total # of reviews for unpaid
total_unpaid_reviews = novine_df.count()
total_unpaid_reviews

43745

In [24]:
#Total 5-star reviwes for paid
paid_5star = vine_df.filter(vine_df.star_rating == 5).count()
paid_5star

430

In [32]:
#Total 5-star reviwes for unpaid
unpaid_5star = novine_df.filter(novine_df.star_rating == 5).count()
unpaid_5star

19233

In [33]:
#% of 5-star reviews paid
paid_5star_percent = (paid_5star / total_paid_reviews) * 100
paid_5star_percent

44.375644994840044

In [34]:
#% of 5-star reviews unpaid
unpaid_5star_percent = (unpaid_5star / total_unpaid_reviews) * 100
unpaid_5star_percent

43.96616756200709